In [ ]:
import re
import time
import random
import pandas as pd
from selenium import webdriver
from pyproj import Proj, Transformer
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(f"https://new.land.naver.com/offices?ms=35.1199095,126.8368983,15&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL")


In [ ]:
pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
cnt = 0

for i in pin_elements:
    # 숫자만 추출 (예: "10\n개 매물" → 10)
    numbers = re.findall(r'\d+', i.text)
    if numbers:
        cnt += int(numbers[0])  # 첫 번째 숫자만 사용

print(cnt)

In [ ]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains


while True:
    pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")

    if pin_elements:

        while True:
            target = random.choice(pin_elements)

            target_size = target.size
            target_location = target.location
            target_x = target_location['x'] + target_size['width'] // 2
            target_y = target_location['y'] + target_size['height'] // 2

            # 기존 조건도 유지
            if target_x < 380 or target_y < -30:
                continue
            else:
                try:
                    target.click()
                    break
                except:
                    pass
            
            # 클릭
        time.sleep(0.5)  # 클릭 후 렌더링 시간 고려

        # 드래그를 위한 좌표 계산
        map_wrap = driver.find_element(By.CSS_SELECTOR, "div.wrap")  # 필요한 경우 정확한 selector로 수정


        # 중심 좌표 계산
        map_size = map_wrap.size
        map_location = map_wrap.location
        map_center_x = map_location['x'] + map_size['width'] // 2
        map_center_y = map_location['y'] + map_size['height'] // 2

        # target 좌표 계산
        target_size = target.size
        target_location = target.location
        target_center_x = target_location['x'] + target_size['width'] // 2
        target_center_y = target_location['y'] + target_size['height'] // 2

        # 이동할 거리 계산 (map 중심 - target 중심)
        offset_x = map_center_x - target_center_x
        offset_y = map_center_y - target_center_y

        # 드래그 수행
        actions = ActionChains(driver)
        actions.move_to_element(target).click_and_hold().move_by_offset(offset_x, offset_y).release().perform()

        time.sleep(1)  # 드래그 후 렌더링 시간 고려
        
        url = driver.current_url  # 먼저 문자열로 저장
        coord_str = url[url.find("ms=")+3 : url.find(",18&a")]  # "ms=" 다음부터 좌표 추출
        coords = coord_str.split(',')  # ['위도', '경도']

        if any((unique_coords['위도'] == coords[0]) & (unique_coords['경도'] == coords[1])):
            pass  # 같은 좌표가 이미 있음 → 건너뛰기
        else:

            while True:
                target.click()
                target_items = driver.find_elements(By.CSS_SELECTOR, "div.item_inner")
                try:
                    body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
                    while True:
                        body.send_keys(Keys.END)
                        time.sleep(1)
                        driver.find_element(By.CSS_SELECTOR, 'div.loader')  # 로딩 대기
                except:
                    pass
                target_items = driver.find_elements(By.CSS_SELECTOR, "div.item_inner")
                time.sleep(2)
                
                if int(target.text[0]) != len(target_items):
                    # target.click()
                    pass
                elif int(target.text[0]) == len(target_items):
                    break

            target_items = driver.find_elements(By.CSS_SELECTOR, "div.item_title")
            for item in target_items:

                item.click()
                time.sleep(0.5)

                ty = driver.find_elements(By.CSS_SELECTOR, "h4.info_title")
                title = ty[1].text

                pay = driver.find_element(By.CSS_SELECTOR, "div.info_article_price ")
                price = pay.text

                rows = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")

                data = {}
                stop_flag = False
                for row in rows:
                    if stop_flag:
                        break
                    ths = row.find_elements(By.CSS_SELECTOR, "th")
                    tds = row.find_elements(By.CSS_SELECTOR, "td")

                    for i in range(len(ths)):
                        key = ths[i].text.strip()
                        value = tds[i].text.strip() if i < len(tds) else ""

                        data[key] = value

                        if key == "건물번호":
                            stop_flag = True
                            break

                # 제목과 가격 추가
                data["제목"] = title
                data["가격"] = price
                data['위도'] = coords[0]
                data['경도'] = coords[1]
                
                df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
                unique_coords = df[['위도', '경도']].drop_duplicates()

            c_btns = driver.find_elements(By.CSS_SELECTOR, "button.btn_close")

            for btn in c_btns:
                label = btn.get_attribute("aria-label")
                if label == "상세페이지 닫기":
                    btn.click()
                    break  # 클릭한 후 반복 종료


    if cnt == len(unique_coords):
        break


In [ ]:
df[['위도', '경도']].to_csv("위도경도.csv", index=False)